In [ ]:
#!pip install split-folders
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, utils
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import pandas as pd
import os
import shutil
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.metrics import ConfusionMatrixDisplay

import splitfolders
from google.colab import drive
from pathlib import Path

from collections import Counter

# Set Color Pallette
sns.set_palette("viridis")

# Organize image files from a main directory into another directory with subdirectories which are also classes for the images.
def organize_images_by_class(csv_path, source_image_dir, destination_base_dir, image_col='id_code', class_col='diagnosis'):
    try:
        df = pd.read_csv(csv_path)
    except FileNotFoundError:
        print(f"Error: CSV file not found at {csv_path}")
        return
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return

    if image_col not in df.columns or class_col not in df.columns:
        print(f"Error: CSV must contain '{image_col}' and '{class_col}' columns.")
        return

    # Create the base destination directory if it doesn't exist
    os.makedirs(destination_base_dir, exist_ok=True)

    for index, row in df.iterrows():
        image_filename = row[image_col]
        image_filename = image_filename + '.png'
        class_name = str(row[class_col]) # Ensure class_name is a string

        source_path = os.path.join(source_image_dir, image_filename)
        destination_class_dir = os.path.join(destination_base_dir, class_name)
        destination_path = os.path.join(destination_class_dir, image_filename)

        # Create class-specific subfolder if it doesn't exist
        os.makedirs(destination_class_dir, exist_ok=True)

        if os.path.exists(source_path):
            try:
                shutil.copy2(source_path, destination_path)
                print(f"Copied '{image_filename}' to '{destination_class_dir}'")
            except Exception as e:
                print(f"Error copying '{image_filename}': {e}")
        else:
            print(f"Warning: Image file '{image_filename}' not found at '{source_path}'")

# Split Images into Train, Test and Validation folders
def splitImagesIntoTrainTestVal(input_folder, output_folder):
      # Create the output folder if it doesn't exist
      os.makedirs(output_folder, exist_ok=True)

      # Split the data
      # ratio=(train_ratio, validation_ratio, test_ratio)
      splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(0.7, 0.2, 0.1), group_prefix=None)
      print("Image organization complete.")

# Delete Intermediate Directory
def deleteIntermediateFolder(intermediate_dir):
    if os.path.exists(intermediate_dir):
        try:
            shutil.rmtree(intermediate_dir)
            print(f"Deleted intermediate directory: {intermediate_dir}")
        except Exception as e:
            print(f"Error deleting intermediate directory '{intermediate_dir}': {e}")
    else:
        print(f"Intermediate directory '{intermediate_dir}' does not exist.")

# Set up Images in Datasets
def extractDataSet(folder_path, labels, label_mode, image_size, batch_size, seed, shuffle, interpolation):
    return tf.keras.utils.image_dataset_from_directory(
        directory=folder_path,
        labels=labels,
        label_mode=label_mode,
        image_size=image_size,
        batch_size=batch_size,
        interpolation=interpolation,
        seed=seed,
        shuffle=shuffle
    )

# Calculate Class Count in Dataset and print distribution
def showClassDistribution(dataset, datasetName):
    counts = Counter()
    for features, labels in dataset:
        # Convert labels to NumPy array for easier processing
        train_labels_np = labels.numpy()
        counts.update(train_labels_np)

    print({datasetName}, "Class distribution:", dict(counts))
    categories = list(counts.keys())
    values = list(counts.values())
    plt.figure(figsize=(10, 10))
    plt.bar(categories, values)
    plt.title(datasetName + "Class Distribution", fontsize=14)
    plt.xlabel('Categories')
    plt.ylabel('Values')
    plt.xticks(rotation=90) # Rotate x-axis labels for better readability if needed
    plt.tight_layout() # Adjust layout to prevent labels from overlapping

    # Create the directory if it doesn't exist
    if not os.path.exists(DATA_PIPELINE_PATH):
        os.makedirs(DATA_PIPELINE_PATH)

    # Create the full path to the file
    filename = f"{datasetName}_class_distribution.png"
    full_path = os.path.join(DATA_PIPELINE_PATH, filename)

    print(f"  Class Distribution For Dataset {datasetName}: {filename}")
    print(f"Images saved to: {DATA_PIPELINE_PATH}")
    plt.tight_layout()
    # Save file in the directory
    plt.savefig(full_path)
    plt.show()


# Normalize images
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

#Crop images and resize to original size
def crop_image(image, label):
    # Assuming images are in channels_last format (height, width, channels)
    height, width = tf.shape(image)[0], tf.shape(image)[1]
    size = tf.minimum(height, width)
    offset_height = (height - size) // 2
    offset_width = (width - size) // 2
    cropped_image = tf.image.crop_to_bounding_box(
        image,
        offset_height,
        offset_width,
        size,
        size
    )
    # Resize the cropped image back to the original image_size
    resized_image = tf.image.resize(cropped_image, size=(image_size[0], image_size[1]))
    return resized_image, label

# Data Augmentation
def data_augmentation(image, rotation_range, width_shift_range, height_shift_range, shear_range, zoom_range, horizontal_flip, fill_mode):
    x = img_to_array(image)
    x = np.expand_dims(x, axis=0) # Add batch dimension

    # Create an ImageDataGenerator with desired augmentations
    datagen = ImageDataGenerator(
        rotation_range,
        width_shift_range,
        height_shift_range,
        shear_range,
        zoom_range,
        horizontal_flip,
        fill_mode
    )

    # Generate augmented images
    i = 0
    plt.figure(figsize=(10, 5))
    for batch in datagen.flow(x, batch_size=1):
        plt.subplot(1, 5, i + 1)
        plt.imshow(batch[0].astype('uint8'))
        plt.axis('off')
        i += 1
        if i % 5 == 0: # Generate 5 augmented images for demonstration
            break
    plt.show()

    return datagen.flow(1, batch_size=32)

# Preprocess Images
def preprocess_images(image, label):
    normalize_img(image, label)
    crop_image(image, label)
    return image, label

# Preview Images
def previewImages(dataset, datasetName, path_to_save_images):
    plt.figure(figsize=(10, 10))
    print(len(dataset.take(1)))
    class_names = ["Class 0", "Class 1", "Class 2", "Class 3", "Class 4"]

    for images, labels in dataset.take(1):
      for i in range(9):
        if i < images.shape[0]:
          ax = plt.subplot(3, 3, i + 1)
          plt.imshow(images[i].numpy().astype("uint8"))
          plt.title(class_names[labels[i]])
          plt.axis("off")

    # Create the directory if it doesn't exist
    if not os.path.exists(path_to_save_images):
        os.makedirs(path_to_save_images)

    # Create the full path to the file
    filename = f"{datasetName}_previewImages.png"
    full_path = os.path.join(path_to_save_images, filename)

    print(f"  Preview Images For Dataset {datasetName}: {filename}")
    print(f"Images saved to: {path_to_save_images}")

    plt.tight_layout()
    # Save file in the directory
    plt.savefig(full_path)
    plt.show()

# Save Pre-processed dataset
def saveImagesToFolder(dataset, path_to_save_images):
    os.makedirs(path_to_save_images, exist_ok=True)

    for i, (images, labels) in enumerate(dataset):
        for j in range(images.shape[0]):
            img_array = images[j].numpy()
            label_val = labels[j].numpy()
            os.makedirs(os.path.join(path_to_save_images / f"{label_val}"), exist_ok=True)
            filename = f"image_batch{i}_idx{j}_label{label_val}.png"
            print("Saving file {filename} to path: ", path_to_save_images / f"{label_val}")
            tf.keras.utils.save_img(os.path.join(path_to_save_images / f"{label_val}", filename), img_array)

    print(f"Images saved to: {path_to_save_images}")

# --- Pipeline Begin ---

if __name__ == "__main__":

    # Data Path
    DATA_PIPELINE_PATH = '/data/processed'

    # Organize images into folders by class/label
    csv_file_path = '/healthcare/retinal_labels.csv'  # Path to your CSV file
    image_source_directory = '/data/raw/retinal_images_256'  # Directory where your images are currently located
    destination_root_directory = '/data/raw/Labeled_Images'  # Parent directory for your new subfolders

    organize_images_by_class(csv_file_path, image_source_directory, destination_root_directory)
    print("Image organization complete.")

    #Split Images into Train, Test and Val folders

    # Define the input and output directories
    input_folder = '/data/raw/Labeled_Images'  # Replace with the path to your original image folder
    output_folder = '/data/raw/Train_Test_Val_Split_Images' # Replace with your desired output directory

    splitImagesIntoTrainTestVal(input_folder, output_folder)

    # Delete intermediate folder
    intermediate_dir = '/data/raw/Labeled_Images'
    deleteIntermediateFolder(intermediate_dir)

    # Load datasets
    train_image_dir = '/data/raw/Train_Test_Val_Split_Images/train'
    test_image_dir = '/data/raw/Train_Test_Val_Split_Images/test'
    val_image_dir = '/data/raw/Train_Test_Val_Split_Images/val'

    # Set dataset parameters
    image_size = (512, 512)
    img_height = 512
    img_width = 512
    batch_size = 32
    seed = 0
    validation_split_ratio = 0 # 0% for validation, since the images are already split into train, test and val
    interpolation="bilinear"
    labels="inferred"
    label_mode='int'
    shuffle=True

    # Set datasets
    train_ds = extractDataSet(train_image_dir, labels, label_mode, image_size, batch_size, seed, True, interpolation)
    test_ds = extractDataSet(test_image_dir, labels, label_mode, image_size, batch_size, seed, True, interpolation)
    val_ds = extractDataSet(val_image_dir, labels, label_mode, image_size, batch_size, seed, False, interpolation)

    # Review Dataset Specs
    print("Train Dataset Spec: ", train_ds.element_spec)
    print("Test Dataset Spec: ", test_ds.element_spec)
    print("Val Dataset Spec: ", val_ds.element_spec)

    # ShowClassDistributions By Dataset
    showClassDistribution(train_ds, "Train_DataSet")
    showClassDistribution(test_ds, "Test_DataSet")
    showClassDistribution(val_ds, "Val_DataSet")

    # Preprocess dataset - Normalize and Crop images
    rotation_range = 0.2
    width_shift_range = 0.2
    height_shift_range = 0.2
    shear_range = 0.2
    zoom_range = 0.2
    horizontal_flip = True
    fill_mode = 'nearest'
    data_augmentation_needed = True

    preprocessed_train = train_ds.map(preprocess_images)
    #preprocessed_test = test_ds.map(preprocess_images)
    preprocessed_val = val_ds.map(preprocess_images)

    # Preview images
    previewImages(preprocessed_train, "Train_DataSet", DATA_PIPELINE_PATH)
    previewImages(test_ds, "Test_DataSet", DATA_PIPELINE_PATH)
    previewImages(preprocessed_val, "Val_DataSet", DATA_PIPELINE_PATH)

    # Save images to pipeline
    saveImagesToFolder(preprocessed_train, DATA_PIPELINE_PATH / 'train')
    saveImagesToFolder(test_ds, DATA_PIPELINE_PATH / 'test')
    saveImagesToFolder(preprocessed_val, DATA_PIPELINE_PATH / 'val')

    # --- Pipeline End --